In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
#
# ASE is a very convenient module for setting up simulations on molecules and 
# bulk materials
#
from ase.io import read, write
from ase.build import molecule
from ase.visualize import view
from ase.calculators.espresso import Espresso

In [ ]:
os.environ['ASE_ESPRESSO_COMMAND'] = "mpirun -np 4 /Users/olivieroandreussi/PWSCF/qe-git/bin/pw.x -in PREFIX.pwi > PREFIX.pwo"
os.environ['OMP_NUM_THREADS'] = "1"

In [ ]:
eV2Ry = 13.605662285137 # energy conversion factor
eV2kcal_mol = 23.0609 # energy conversion factor
bohr2ang = 0.5291772 # length conversion factor
ang2bohr = 1./bohr2ang

In [ ]:
atoms = molecule('H2O')
view(atoms, viewer="x3d")

In [ ]:
atoms.set_cell(15. * np.identity(3))
atoms.set_pbc((True, True, True))
atoms.center()

In [ ]:
pseudopotentials = {
    "H":"H.pbe-rrkjus.UPF",
    "O":"O.pbe-rrkjus.UPF"
}

In [ ]:
input_data = {
    'control': {
        'restart_mode': 'from_scratch',
        'pseudo_dir': '/Users/olivieroandreussi/PWSCF/qe-7.2/pseudo',
        'calculation': 'relax',
        'prefix': 'H2O_vacuum'
    },
    'system': {
        'ecutwfc': 30,
        'ecutrho': 300
    },
    'electrons': {
        'diagonalization':'david',
        'conv_thr': 1.0e-8, 
        'mixing_beta': 0.4
    }
} 

calc = Espresso(
    pseudopotentials=pseudopotentials,
    tstress=True, tprnfor=True, 
    input_data = input_data,
    kpts=(1,1,1),
    koffset=(0,0,0))

atoms.calc = calc

In [ ]:
energy = atoms.get_potential_energy()
print(f"Energy in vacuum = {energy:.3f} eV")

In [ ]:
relaxed_atoms = read('./espresso.pwo',format='espresso-out')

In [ ]:
atoms.positions

In [ ]:
relaxed_atoms.positions

# Use ASE for Geometry Optimization

In [ ]:
from ase.optimize import BFGS

In [ ]:
atoms = molecule('H2O')
atoms.set_cell(15. * np.identity(3))
atoms.set_pbc((True, True, True))
atoms.center()

In [ ]:
input_data = {
    'control': {
        'restart_mode': 'from_scratch',
        'pseudo_dir': '/Users/olivieroandreussi/PWSCF/qe-7.2/pseudo',
        'calculation': 'scf',
        'prefix': 'H2O_vacuum'
    },
    'system': {
        'ecutwfc': 30,
        'ecutrho': 300
    },
    'electrons': {
        'diagonalization':'david',
        'conv_thr': 1.0e-8, 
        'mixing_beta': 0.4
    }
} 

calc = Espresso(
    pseudopotentials=pseudopotentials,
    tstress=True, tprnfor=True, 
    input_data = input_data,
    kpts=(1,1,1),
    koffset=(0,0,0))

atoms.calc = calc

In [ ]:
optimization = BFGS(atoms,trajectory='H2O-opt.traj')
optimization.run(fmax=0.001)

In [ ]:
atoms.positions

# Use ASE for Phonons Calculations

In [ ]:
from ase.phonons import Phonons

In [ ]:
# Phonon calculator
N = 1
ph = Phonons(atoms, calc, supercell=(N, N, N), delta=0.05)
ph.run()
